In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert-test/merged_test_data_with_bert.csv
/kaggle/input/train-bert/merged_train_data_with_bert.csv
/kaggle/input/train-csv/train (1).csv
/kaggle/input/test-csv/test.csv


In [74]:
# imports

import pandas as pd
import numpy as np
import os
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC

# Reading and Splitting data
The train.csv (including the features from the bert model) was read into `train_df` variable

The test.csv was read into `eval_df` variable

Split the train_df into `train_df` which contains the features and `train_df_y` which contains the phishy label

In [75]:
# reading and splitting the data
train_df = pd.read_csv("/kaggle/input/train-bert/merged_train_data_with_bert.csv")
eval_df = pd.read_csv("/kaggle/input/test-csv/test.csv")

train_df_y = train_df.pop('phishy')

# Preprocessing

## Adding variables to the dataframe
The following variables were added: 
- `url_len` (numeric): the length of the url
- `url_nums` (numeric): the number of digits in the url
- `num_slashes` (numeric): the number of slashes in the url
- `num_periods`(numeric): the number of periods in the url
- `num_hyphens` (numeric): the number of hyphens in the url
- `contains_com` (boolean): whether the url contains .com
- `contains_org` (boolean): whether the url contains .org
- `contains_gov` (boolean): whether the url contains .gov
- `contains_www` (boolean) : whether the url contians www.
- `contains_html` (boolean): whether the url contains .html
- 

## Scaling


In [77]:
# # Preprocessing: adding new variables based on the url
# commented out because already added

# def count_char(s, char):
#     return s.count(char)

# def count_num(s):
#     return sum(c.isdigit() for c in s)

# # adding the length of the url
# train_df['url_len'] = train_df['url'].apply(len)

# # adding the number of numbers in url ##
# train_df['url_nums'] = train_df['url'].apply(lambda x: count_num(x))

# # the number of slashes in the url
# char_to_count = '/'
# train_df['num_slashes'] = train_df['url'].apply(lambda x: count_char(x,char_to_count))

# # the number of periods in the url
# char_to_count = '.'
# train_df['num_periods'] = train_df['url'].apply(lambda x: count_char(x,char_to_count))

# # the number of hyphens in the url
# char_to_count = '-'
# train_df['num_hyphens'] = train_df['url'].apply(lambda x: count_char(x,char_to_count))

# # boolean: contains.com
# train_df['contains_com'] = train_df['url'].str.contains('.com')

# # boolean: contains .gov
# train_df['contains_gov'] = train_df['url'].str.contains('.gov')

# # boolean: contains .org
# train_df['contains_org'] = train_df['url'].str.contains('.org')

# # boolean: contains .html ##
# train_df['contains_html'] = train_df['url'].str.contains('.html')

# # boolean: contains www. ##
# train_df['contains_www'] = train_df['url'].str.contains('www.')


In [78]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df_y, test_size=0.3, random_state=42)

In [79]:
# Preprocessing 2: removing identifier columns and scaling numerical values

# Removing identifier columns and url column
X_train = X_train.drop(['id', 'url'], axis=1)


# Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
boolean_cols = X_train.select_dtypes(include=['bool']).columns

# Define the preprocessing steps for numerical and categorical data
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a ColumnTransformer to apply the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('bool', 'passthrough', boolean_cols)
    ])


X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)



In [82]:
model = Sequential()
#model.add(Dense(128, input_dim=779, activation='relu'))
model.add(Dense(32, activation='relu'))# Input layer with 3 input features
model.add(Dense(16, activation='relu'))# Hidden layer
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))             # Output layer for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01),
              loss=BinaryCrossentropy(),
              metrics=['accuracy', AUC()])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
history = model.fit(X_train_processed, y_train, epochs=30, batch_size=7, validation_split=0.3, verbose=1)


Epoch 1/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9806 - auc_5: 0.9893 - loss: 0.3610 - val_accuracy: 0.9959 - val_auc_5: 0.9979 - val_loss: 0.0277
Epoch 2/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9835 - auc_5: 0.9961 - loss: 0.0815 - val_accuracy: 0.9949 - val_auc_5: 0.9970 - val_loss: 0.0386
Epoch 3/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9906 - auc_5: 0.9966 - loss: 0.0547 - val_accuracy: 0.9949 - val_auc_5: 0.9981 - val_loss: 0.0250
Epoch 4/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9938 - auc_5: 0.9944 - loss: 0.0409 - val_accuracy: 0.9939 - val_auc_5: 0.9980 - val_loss: 0.0274
Epoch 5/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9970 - auc_5: 0.9990 - loss: 0.0185 - val_accuracy: 0.9970 - val_auc_5: 0.9990 - val_loss: 0.0173
Epoch 6/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9952 - auc_5: 0.9976 - loss: 0.0253 - val_accuracy: 0.9959 - val_auc_5: 0.9998 - val_loss: 0.0148
Epoch 7/30
329/3

In [84]:
results = model.evaluate(X_test_processed, y_test)
print(f'Test Loss: {results[0]}')
print(f'Test Accuracy: {results[1]}')
print(f'Test AUC: {results[2]}')


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9962 - auc_5: 0.9999 - loss: 0.0134
Test Loss: 0.022165147587656975
Test Accuracy: 0.9957386255264282
Test AUC: 0.9992095828056335


In [86]:
# Preprocessing: 

test_df = pd.read_csv('/kaggle/input/bert-test/merged_test_data_with_bert.csv')

results_df = test_df.pop('id')
# Removing identifier columns and url column
test_df = test_df.drop([ 'url'], axis=1)


# Identify numerical and categorical columns
numerical_cols = test_df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = test_df.select_dtypes(include=['object']).columns
boolean_cols = test_df.select_dtypes(include=['bool']).columns

# Define the preprocessing steps for numerical and categorical data
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a ColumnTransformer to apply the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('bool', 'passthrough', boolean_cols)
    ])


test_df_processed = preprocessor.fit_transform(test_df)

In [90]:
probabilities = model.predict(test_df_processed)
predictions = (probabilities >= 0.5).astype(int)
predictions = np.concatenate(predictions)
final_results = pd.DataFrame({
    'id': results_df,
    'pred': predictions
})

print(final_results)
final_results.to_csv('/kaggle/working/preds.csv', index=False)

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
            id  pred
0     7443817b     0
1     f7223190     0
2     228dd2b4     1
3     3f6cdff2     0
4     665221be     0
...        ...   ...
1144  6558e4c0     1
1145  b825b896     1
1146  fa8d0b96     1
1147  a6b60922     0
1148  7ac68919     1

[1149 rows x 2 columns]
